In [1]:
import math
import random

from pythreejs import *
from IPython.display import display

import numpy as np

In [2]:
scene = Scene(
    background='#000000'
)

camera = PerspectiveCamera(
    position=[10, 10, 10], 
    fov=50, 
    aspect=1.5
)

camera.lookAt([2, 1, 0])

renderer = Renderer(
    camera=camera, 
    scene=scene, 
    controls=[OrbitControls(controlling=camera)], 
    width=800, 
    height=600
)

In [34]:
def make_random_tracks(number_of_tracks):

    tracks = []
    
    for i in range(0,number_of_tracks+1):
        pt = 1.0 + random.random()*(10-2.0)
        charge = 1 if random.random() > 0.5 else -1
        eta = ((2*random.random())-1)*2.4
        phi = ((2*random.random())-1)*math.pi

        print(f'pt, eta, phi, charge: {pt}, {eta}, {phi}, {charge}')
        
        tracks.append(
            (pt,eta,phi,charge)
        )

    return tracks


def make_helix_points(track):

    pt = track[0]
    eta = track[1]
    phi = track[2]
    charge = track[3]
    
    vertex = (0,0,0)
                                                                                                                                                                                  
    Bz = 3.8
    radius = (pt / (math.fabs(charge)*Bz))

    centerX = vertex[0] + radius*charge*math.sin(phi)
    centerY = vertex[1] - radius*charge*math.cos(phi)
    
    pz = pt*math.sinh(eta)
    phi0 = math.atan2(vertex[1] - centerY, vertex[0] - centerX)
    omega = (charge*Bz) / pt
    pitch = pz / (pt*math.fabs(omega))

    print(f'pitch: {pitch}')
    
    maxAngle = 4*math.pi
    numPoints = 1000
    points = []

    for i in range(0, numPoints+1):

        t = (i/numPoints)*maxAngle
        x = centerX + radius*math.cos(phi0 + np.sign(omega)*t)
        y = centerY + radius*math.sin(phi0 + np.sign(omega)*t)
        z = vertex[2] + t*pitch
        r = math.sqrt(x*x +y*y)                                                                                                                                                                       
        
        if math.fabs(z) > 3 or r > 1.24:
            break
        
        points.append((float(x),float(y),float(z)))

    points = np.array(points)
    return charge, points

def make_eb():

    geometry = CylinderGeometry(
        radiusTop=1.12, 
        radiusBottom=1.24, 
        height=6.0, 
        radialSegments=64, 
        heightSegments=1, 
        openEnded=True, 
        thetaStart=0, 
        thetaLength=2*math.pi
    )

    eb = Mesh(
        geometry,
        MeshBasicMaterial(
            color='#7fccff',
            wireframe=True,
            transparent=True,
            opacity=0.2
        )
    )

    eb.rotateX(math.pi/2)
    eb.name = 'EB'
    
    return eb

In [35]:
def add_axes():
    
    axes = []

    length = 1.0
    
    x_axis = ArrowHelper(
        dir=[1, 0, 0],
        origin=[0, 0, 0],
        length=length,
        color='#ff0000',
        headLength=0.2,
        headWidth=0.1
    )
    x_axis.name = 'X_Axis'
    axes.append(x_axis)
        
    y_axis = ArrowHelper(
        dir=[0, 1, 0],
        origin=[0, 0, 0],
        length=length,
        color='#00ff00',
        headLength=0.2,
        headWidth=0.1
    )
    y_axis.name = 'Y_Axis'
    axes.append(y_axis)
        
    z_axis = ArrowHelper(
        dir=[0, 0, 1],
        origin=[0, 0, 0],
        length=length,
        color='#0000ff',
        headLength=0.2,
        headWidth=0.1
    )
    z_axis.name = 'Z_Axis'
    axes.append(z_axis)
        
    return axes


In [36]:
scene.children = []

tracks = make_random_tracks(10)

for track in tracks:
    charge, points = make_helix_points(track)
    points = np.array(points)

    geometry = BufferGeometry(attributes={
        'position': BufferAttribute(points, normalized=False)
    })

    if np.sign(charge) == -1:
        color = '#ff00ff'
    else:
        color = '#00ffff'
  
    material = LineBasicMaterial(color=color)
    line = Line(geometry, material)
    scene.add(line)

pt, eta, phi, charge: 2.5306918709430564, -0.09656829685188714, -2.3517912508908796, 1
pt, eta, phi, charge: 8.848318171585976, 1.8705687273810563, -1.7751124271437568, -1
pt, eta, phi, charge: 7.912132894348128, -2.279258109285879, 0.571029959881251, 1
pt, eta, phi, charge: 4.876346451671932, 2.1705945403327473, 2.204646051980478, -1
pt, eta, phi, charge: 2.540619815615343, -1.0390953850377875, -1.2455851746236082, 1
pt, eta, phi, charge: 4.008918728398319, -0.9417488465110395, 2.9503976943706793, -1
pt, eta, phi, charge: 6.081087781043461, -2.2706865569557406, -1.2074001699280537, -1
pt, eta, phi, charge: 3.5565974869757255, 0.10114489319865605, -0.751763541306638, 1
pt, eta, phi, charge: 6.0391721592808585, -1.2960033205056758, 0.5153195055363894, -1
pt, eta, phi, charge: 7.494834033985401, -0.7242985608343797, 1.2406724467668577, -1
pt, eta, phi, charge: 6.645959041075402, -1.8900577879305338, -0.6998921374010167, -1
pitch: -0.06441174030005879
pitch: 7.378975174210488
pitch: -10.0

In [37]:
scene.add(make_eb())
scene.add(add_axes())
display(renderer)

Renderer(camera=PerspectiveCamera(aspect=1.5, position=(-0.3885850531431821, 0.5049229515369861, -6.1763825528…